In [25]:
# coding: utf-8

"""
title: R0 estimation in the World
date: 2020/03/22
author: okimebarun
url: https://github.com/okimebarun/01_COVID19_analysis
url: https://qiita.com/oki_mebarun
"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import locale

In [26]:
font = {'family' : 'IPAexGothic'}
plt.rc('font', **font)

In [27]:
##########################################################################################

In [28]:
def makeCalcFrame(days):
    t_1 = pd.Timestamp(2020,1,24) # 計算開始日
    td = pd.Timedelta('1 days')
    #
    npd = [[t_1 + td * i, 0, 0, 0 ] for i in range(0,days)]
    df1 = pd.DataFrame(npd)
    df1.columns = ['date', 'Ppre','Pat', 'R0']
    #
    return df1

In [29]:
def mergeCalcFrame(df1, df2):
    return pd.merge(df1, df2, on='date', how='left').fillna(0)

In [30]:
def calcR0(df, keys):
    lp = keys['lp']
    ip = keys['ip']
    nrow = len(df)
    getP = lambda s: df.loc[s, 'P'] if s < nrow else np.NaN
    for t in range(nrow):
        df.loc[t, 'Ppre'] = sum([ getP(s) for s in range(t+1, t + ip + 1)])
        df.loc[t, 'Pat' ] = getP(t + lp + ip)
        if df.loc[t, 'Ppre'] > 0:
            df.loc[t, 'R0'  ] = ip * df.loc[t, 'Pat'] / df.loc[t, 'Ppre']
        else:
            df.loc[t, 'R0'  ] = np.NaN
    return df

In [31]:
def showResult(df, title):
    # R0=1
    ptgt = pd.DataFrame([[df.iloc[0,0],1],[df.iloc[len(df)-1,0],1]])
    ptgt.columns = ['date','target']
    # show R0
    plt.rcParams["font.size"] = 12
    ax = df.plot(title=title,x='date',y='R0', figsize=(10,7))
    ptgt.plot(x='date',y='target',style='r--',ax=ax)
    ax.grid(True)
    ax.set_ylim(0,)
    plt.show()
    fig = ax.get_figure()
    fig.savefig("R0_{}.png".format(title))

In [32]:
##########################################################################################

In [33]:
def readCsvOfWorldArea(area : None):
    # 下記URLよりダウンロード
    # https://hackmd.io/@covid19-kenmo/dataset/https%3A%2F%2Fhackmd.io%2F%40covid19-kenmo%2Fdataset
    fcsv = u'World-COVID-19.csv'
    df = pd.read_csv(fcsv, header=0, encoding='sjis', parse_dates=[u'日付'])
    # 日付, 対象国を抽出
    if area is not None:
        df1 = df.loc[:,[u'日付',area]]
    else:
        df1 = df.loc[:,[u'日付',u'世界全体の感染者']]        
    df1.columns = ['date','Psum']
    ## 累積⇒日次に変換
    df2 = df1.copy()
    df2.columns = ['date','P']
    df2.iloc[0,1] = 0
    ## 文字列⇒数値
    getFloat = lambda e: float('{}'.format(e).replace(',',''))
    ## 差分計算
    for i in range(1,len(df1)):
        df2.iloc[i, 1] = getFloat(df1.iloc[i, 1]) - getFloat(df1.iloc[i-1, 1] )
    ##
    return df2

In [34]:
def R0inWorldArea(area, bShow:True):
    keys = {'lp':5, 'ip':8 }
    df1 = makeCalcFrame(60) # 60 days
    df2 = readCsvOfWorldArea(area)
    df = mergeCalcFrame(df1, df2)
    df = calcR0(df, keys)
    if bShow:
        showResult(df, u'COVID-19 R0 ({})'.format(area))
    return df

In [35]:
def getWorldAreaList():
    fcsv = u'World-COVID-19.csv'
    df = pd.read_csv(fcsv, header=0, encoding='sjis', parse_dates=[u'日付'])
    arealist = df.columns[1:]
    return arealist

arealist = getWorldAreaList()

In [36]:
def showResult2(ax, df, area):
    # show R0
    plt.rcParams["font.size"] = 12
    df1 = df.rename(columns={'R0':area})
    df1.plot(x='date',y=area, ax=ax)

def showResult3(dflist, title):
    # R0=1
    dfs = dflist[0][0]
    ptgt = pd.DataFrame([[dfs.iloc[0,0],1],[dfs.iloc[len(dfs)-1,0],1]])
    ptgt.columns = ['date','target']
    ax = ptgt.plot(title='COVID-19 R0', x='date',y='target',style='r--', figsize=(10,8))
    ax.set_yscale("symlog", linthreshy=1)
    #
    for df, label in dflist:
        showResult2(ax, df, label)
    #
    ax.grid(True)
    ax.set_ylim(0,)
    plt.show()
    fig = ax.get_figure()
    fig.savefig("R0_{}.png".format(title))

In [ ]:
#　
title = '爆発的感染が観測された地域'
arealist = ['中国本土の感染者数', 'イタリア', '米国', 'スペイン', 'イラン','韓国']
dflist = [ [R0inWorldArea(area, False), area] for area in arealist]
showResult3(dflist, title)
#
title = 'ヨーロッパ'
arealist = ['イタリア',  'スペイン', 'ドイツ', 'フランス',  'スイス', '英国','オーストリア',
            'ベルギー','ノルウェー','スウェーデン','デンマーク','ポルトガル']
dflist = [ [R0inWorldArea(area, False), area] for area in arealist]
showResult3(dflist, title)
#
title = 'アジア周辺で比較的感染が抑制されている地域'
arealist = [   '台湾','日本','香港','シンガポール']
dflist = [ [R0inWorldArea(area, False), area] for area in arealist]
showResult3(dflist, title)
#
title = '今後感染拡大が懸念される地域'
arealist = [  '米国','豪州','マレーシア', 'ブラジル','トルコ', 'イスラエル','インドネシア']
dflist = [ [R0inWorldArea(area, False), area] for area in arealist]
showResult3(dflist, title)

In [ ]:
#
title = 'ヨーロッパ+推定'
arealist = ['イタリア',  'スペイン', 'ドイツ', 'フランス',  'スイス', '英国','オーストリア',
            'ベルギー','ノルウェー','スウェーデン','デンマーク','ポルトガル']
dflist = [ [R0inWorldArea(area, False), area] for area in arealist]
#
# R0=1
dfs = dflist[0][0]
ptgt = pd.DataFrame([[dfs.iloc[0,0],1],[dfs.iloc[len(dfs)-1,0],1]])
ptgt.columns = ['date','target']
ax = ptgt.plot(title='COVID-19 R0', x='date',y='target',style='r--', figsize=(10,8))
ax.set_yscale("symlog", linthreshy=1)

# Rt= Rt0 * pow(2.0,  -(t-t0)/T)
Rt0 = 30
Th = 7.5
Rt1 = Rt0 * pow(2.0, -(55. - 20. ) / Th) 
rest = pd.DataFrame([[dfs.iloc[20,0],Rt0],[dfs.iloc[55,0],Rt1]])
rest.columns = ['date','Rt_estimation']
rest.plot(x='date',y='Rt_estimation', ax=ax, style='b:')
#
for df, label in dflist:
    showResult2(ax, df, label)
#
ax.grid(True)
ax.set_ylim(0,)
plt.show()
fig = ax.get_figure()
fig.savefig("R0_{}.png".format(title))

In [39]:
dfs = dflist[0][0]
adv = 17
print("t:{}, Rt:{}".format(dfs.iloc[20+adv,0], Rt0 * pow(2.0, -(20.+adv -20. ) / Th) ))
adv = 37
print("t:{}, Rt:{}".format(dfs.iloc[20+adv,0], Rt0 * pow(2.0, -(20.+adv -20. ) / Th) ))
adv = 62
dt = pd.Timedelta('1 days')
print("t:{}, Rt:{}".format(dfs.iloc[20,0] + adv*dt, Rt0 * pow(2.0, -(20.+adv -20. ) / Th) ))
adv = 77
dt = pd.Timedelta('1 days')
print("t:{}, Rt:{}".format(dfs.iloc[20,0] + adv*dt, Rt0 * pow(2.0, -(20.+adv -20. ) / Th) ))
adv = 92
dt = pd.Timedelta('1 days')
print("t:{}, Rt:{}".format(dfs.iloc[20,0] + adv*dt, Rt0 * pow(2.0, -(20.+adv -20. ) / Th) ))

t:2020-03-01 00:00:00, Rt:6.234284221070908
t:2020-03-21 00:00:00, Rt:0.9818382401443373
t:2020-04-15 00:00:00, Rt:0.09741069095423288
t:2020-04-30 00:00:00, Rt:0.02435267273855822
t:2020-05-15 00:00:00, Rt:0.006088168184639555
